In [10]:
import os
import numpy as np
import random
import glob
from datetime import datetime
import logging
import sys
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras import backend as K

In [11]:
# Limpa a sessão antes de construir o modelo
K.clear_session()

# Configurações de GPU
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Define a política de precisão mista
set_global_policy('mixed_float16')

In [12]:
def parse_file_pointer(fp, tam):
    lines = [ll.strip() for ll in fp]
    ii = 0
    labels = []
    res = []
    cli = []
    numLinhas = 0
    while ii < len(lines):
        line = lines[ii]
        #contando o numero de vertices do grafo
        if "cliqueatual" not in line:
            ii += 1
            numLinhas += 1
            continue

        #pegando a clique atual
        if ii+1 >= len(lines):
            break
        line = line[3:]
        spritado = line.split()
        clique = [int(elem) for elem in spritado[1:]]
        if(numLinhas < tam):
            dif = tam - numLinhas
            clique.extend([0]*dif)
        cli.append(clique)

        #criando o vetor de movimento
        line = lines[ii+1]
        sp = line.split()
        mv = int(sp[-1])
        label = [0] * tam
        label[mv] = 1
        labels.append(label)

        #lendo o grafo
        cells = []
        for tt in range(numLinhas, 0, -1):
            cell_line = lines[ii - tt][3:]
            cells.extend([int(float(cc)) for cc in cell_line.split(", ")])
            if(numLinhas < tam):
                dif = tam - numLinhas
                cells.extend([0]*dif)
        while len(cells) < tam * tam:
            cells.extend([0]*tam)

        #cells = np.reshape(cells,((tam,  -1)))
        #cells = np.transpose(cells)
        #cells = np.reshape(cells, -1)
        res.append(cells)
        ii += (numLinhas+2)
    labels_v = list(range(len(labels),0, -1))
    return (res, cli, labels, labels_v)

In [13]:
def parse_dir(ddir, tam):
    res = []
    cli = []
    labels = []
    labels_v = []
    random.seed(42)
    files = sorted([os.path.basename(ii) for ii in glob.glob("{0}/*.dimacs".format(ddir))])
    random.shuffle(files)
    random.seed()
    i = 0
    for ff in files:
        with open(os.path.join(ddir,ff), 'r') as fp:
            rr, cc, ll, ll_v = parse_file_pointer(fp, tam)
            res.extend(rr)
            cli.extend(cc)
            labels.extend(ll)
            labels_v.extend(ll_v)
        i+=1
        if i > 100:
            break
    return res, cli, labels, labels_v

In [14]:
class printbatch(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        logging.info("Epoch: "+ str(epoch))
    def on_epoch_end(self, epoch, logs={}):
        logging.info(logs)

In [15]:
class LoggerWriter:
    def __init__(self, level):
        self.level = level

    def write(self, message):
        if message != '\n':
            self.level(message)

    def flush(self):
        self.level(sys.stderr)

In [16]:
def learn(data, clique, labels, tam, output_path, shared_layer_multipliers, layer_multipliers, batch_size, learning_rate):
    shared_layer_multipliers = [x for x in shared_layer_multipliers if x != 0]

    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,)) for _ in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(shared_layer_multipliers)):
        shared_dense = Dense(tam * shared_layer_multipliers[i], activation='relu')
        layer = [shared_dense(l) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(layer_multipliers)):
        layer = Dense((tam+1)*layer_multipliers[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(tam, activation='softmax')(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    # Combine `data` e `clique` ao longo da segunda dimensão
    combined_input = np.array([np.hsplit(np.concatenate([data[i], clique[i]]), tam + 1) for i in range(len(clique))])
    # Dividir a entrada em uma lista de 151 tensores de forma (batch_size, tam)
    combined_input_list = [combined_input[:, i, :] for i in range(combined_input.shape[1])]
    # Converter elementos para tensores
    combined_input_list = [tf.convert_to_tensor(arr) for arr in combined_input_list]
    print(type(combined_input_list))
    print(type(combined_input_list[0]))

    #Verificando as formas
    print(f"combined_input shape: ({len(combined_input_list)}, {len(combined_input_list[0])}, {len(combined_input_list[0][0])})")
    print(f"labels shape: {labels.shape}")

    #treinar modelo
    now = datetime.now()
    model.fit(combined_input_list, labels, epochs=5, batch_size=batch_size, validation_split=0.2, verbose=1,
              callbacks=[printbatch(), EarlyStopping(monitor='val_loss', patience=50, verbose=0),
                         ModelCheckpoint(os.path.join(output_path, "models", "dnn_model_" + str(tam) + "_" + str(now.day) + "." +
                                         str(now.month) + "." + str(now.year) + "_" + "_{epoch:02d}-{val_loss:.2f}.keras"),
                                         monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')])
    model.save(os.path.join(output_path,"dnn_model_" + str(tam) + "_" + str(now.day) + "." +str(now.month) + "." + str(now.year) + "_.keras"))

In [17]:
def learn_value(data, clique, labels, tam, output_path, shared_layer_multipliers, layer_multipliers, batch_size, learning_rate):
    shared_layer_multipliers = [x for x in shared_layer_multipliers if x != 0]

    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,)) for _ in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(shared_layer_multipliers)):
        shared_dense = Dense(tam * shared_layer_multipliers[i], activation='relu')
        layer = [shared_dense(l) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(layer_multipliers)):
        layer = Dense(tam*layer_multipliers[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(1)(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=adam, loss='mse', metrics=['mae'])

    # Combine `data` e `clique` ao longo da segunda dimensão
    combined_input = np.array([np.hsplit(np.concatenate([data[i], clique[i]]), tam + 1) for i in range(len(clique))])
    # Dividir a entrada em uma lista de 151 tensores de forma (batch_size, tam)
    combined_input_list = [combined_input[:, i, :] for i in range(combined_input.shape[1])]
    # Converter elementos para tensores
    combined_input_list = [tf.convert_to_tensor(arr) for arr in combined_input_list]

    #treinar modelo
    now = datetime.now()
    model.fit(combined_input_list, labels, epochs=5, batch_size=batch_size, validation_split=0.2, verbose=1,
              callbacks=[printbatch(),
                         EarlyStopping(monitor='val_loss', patience=50, verbose=0),
                         ModelCheckpoint(os.path.join(output_path, "models", "dnn_value_model_" + str(tam) + "x" + "_" + str(now.day) + "." +
                                        str(now.month) + "." + str(now.year) + "_" + "_{epoch:02d}-{val_loss:.2f}.keras"), monitor='val_loss',
                                         verbose=0, save_best_only=True, save_weights_only=False, mode='auto')])
    model.save(os.path.join(output_path,"dnn_value_model_" +str(tam) + "_" +str(now.day) + "." +str(now.month) +"."+str(now.year)+"_.keras"))

In [18]:
def main():
    output_path = "modelos" #caminha onde é salvo o modelo
    labeled_data_dir = "train_graphs" #caminho dos dados para treinar o modelo
    param_v_a_1 = [4, 3, 2] #camadas compartilhadas rede bound
    param_v_a_2 = [3, 2, 2] #camadas ocultas
    param_p_a_1 = [6, 4, 3] #camadas compartilhadas rede brach
    param_p_a_2 = [9, 6, 2] #camadas ocultas
    param_p_b = 100 #batch size
    param_v_b =  100
    param_p_l = 0.001 #taxa de aprendizado
    param_v_l = 0.001
    tam = 150
    use_value_model = True #se vai treinar a rede de bound

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    #treinar rede branch
    res, clique, labels, labels_v = parse_dir(labeled_data_dir, tam)
    print("leu tudo")
    learn(np.array(res), np.array(clique), np.array(labels), tam, output_path, param_p_a_1, param_p_a_2, param_p_b, param_p_l)
    print("rede 1 treinada")
    #treinar rede bound
    if use_value_model:
        learn_value(np.array(res), np.array(clique), np.array(labels_v), tam, output_path, param_v_a_1, param_v_a_2, param_v_b, param_v_l)
        print("rede 2 treinada")

main()

leu tudo
<class 'list'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
combined_input shape: (151, 1115, 150)
labels shape: (1115, 150)
Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 79s 6s/step - accuracy: 0.0119 - loss: 5.1555 - val_accuracy: 0.0314 - val_loss: 5.2543
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 555ms/step - accuracy: 0.0163 - loss: 5.0832 - val_accuracy: 0.0269 - val_loss: 4.9681
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 572ms/step - accuracy: 0.0160 - loss: 4.9190 - val_accuracy: 0.0269 - val_loss: 4.9525
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 949ms/step - accuracy: 0.0191 - loss: 4.7862 - val_accuracy: 0.0045 - val_loss: 4.9357
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.0341 - loss: 4.6460 - val_accuracy: 0.0045 - val_loss: 4.9563
rede 1 treinada
Epoch 1/5


/home/igor/anaconda3/envs/tcc/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_760', 'keras_tensor_761', 'keras_tensor_762', 'keras_tensor_763', 'keras_tensor_764', 'keras_tensor_765', 'keras_tensor_766', 'keras_tensor_767', 'keras_tensor_768', 'keras_tensor_769', 'keras_tensor_770', 'keras_tensor_771', 'keras_tensor_772', 'keras_tensor_773', 'keras_tensor_774', 'keras_tensor_775', 'keras_tensor_776', 'keras_tensor_777', 'keras_tensor_778', 'keras_tensor_779', 'keras_tensor_780', 'keras_tensor_781', 'keras_tensor_782', 'keras_tensor_783', 'keras_tensor_784', 'keras_tensor_785', 'keras_tensor_786', 'keras_tensor_787', 'keras_tensor_788', 'keras_tensor_789', 'keras_tensor_790', 'keras_tensor_791', 'keras_tensor_792', 'keras_tensor_793', 'keras_tensor_794', 'keras_tensor_795', 'keras_tensor_796', 'keras_tensor_797', 'keras_tensor_798', 'keras_tensor_799', 'keras_tensor_800', 'keras_t

8/9 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 158.1673 - mae: 7.3520

2024-11-30 14:16:12.017325: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_pad_fusion_423', 748 bytes spill stores, 748 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_pad_fusion_422', 284 bytes spill stores, 284 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_maximum_fusion_10', 268 bytes spill stores, 268 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_maximum_fusion_8', 276 bytes spill stores, 276 bytes spill loads



9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 159.4083 - mae: 7.3393  

2024-11-30 14:16:21.537128: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_maximum_fusion_11', 292 bytes spill stores, 292 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_maximum_fusion_9', 276 bytes spill stores, 276 bytes spill loads



9/9 ━━━━━━━━━━━━━━━━━━━━ 99s 7s/step - loss: 160.4010 - mae: 7.3291 - val_loss: 67.3766 - val_mae: 6.8924
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - loss: 36.0221 - mae: 4.7268 - val_loss: 22.3800 - val_mae: 3.7609
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - loss: 14.0268 - mae: 2.9952 - val_loss: 17.9312 - val_mae: 3.3170
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 235ms/step - loss: 11.6649 - mae: 2.6953 - val_loss: 16.5175 - val_mae: 3.3223
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 241ms/step - loss: 10.6425 - mae: 2.6398 - val_loss: 16.2352 - val_mae: 3.2332
rede 2 treinada
